In [1]:
# !curl -o duckdb.jar "https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar"

In [2]:
import duckdb
import pyspark
from pyspark.sql import SparkSession

sc = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/24 14:23:04 WARN Utils: Your hostname, MacBook-Air-de-Olivia.local resolves to a loopback address: 127.0.0.1; using 10.192.23.148 instead (on interface en0)
24/04/24 14:23:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/24 14:23:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
paths = ['../data_lake/compravenda_sup.parquet', '../data_lake/rent_price.parquet', '../data_lake/renda.parquet']

In [5]:
for path in paths:
    RDD = sc.read.parquet(path)
    path = path.split('/')[-1].split('.')[0]
    # conn = duckdb.connect(path)
    # conn.close()
    RDD.write \
        .format("jdbc") \
        .option("url", f"jdbc:duckdb:freshdata.db") \
        .option("dbtable", f"{path}") \
        .option("driver", "org.duckdb.DuckDBDriver") \
        .mode("overwrite") \
        .save()